<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/evaluacion_trabajadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Paso 1: Instalar librerías necesarias
!pip install pandas openpyxl

In [3]:
# Paso 2: Importar librerías
import pandas as pd

# Paso 3: Cargar el archivo Excel
file_path = "Revision_Polizas_Seguros.xlsx"  # Subir el archivo a Colab y usar este nombre
excel_data = pd.ExcelFile(file_path)

# Leer cada hoja en un DataFrame
df_vida = pd.read_excel(excel_data, sheet_name="Vida")
df_salud = pd.read_excel(excel_data, sheet_name="Salud")
df_dental = pd.read_excel(excel_data, sheet_name="Dental")
df_catastrofico = pd.read_excel(excel_data, sheet_name="Catastrófico")

# Paso 4: Consolidar la información
# Crear un DataFrame maestro con todos los trabajadores y sus pólizas
dfs = [df_vida, df_salud, df_dental, df_catastrofico]
df_consolidado = pd.concat(dfs, keys=["Vida", "Salud", "Dental", "Catastrófico"], names=["Póliza"])
df_consolidado.reset_index(level=1, drop=True, inplace=True)
df_consolidado.reset_index(inplace=True)

# Paso 5: Identificar trabajadores y pólizas
# Crear una tabla pivote para ver qué trabajadores tienen qué pólizas
df_pivot = pd.pivot_table(df_consolidado, index=["Nombre", "RUT"], columns="Póliza", values="Valor Prima (UF)", aggfunc="sum", fill_value=0)




In [4]:
df_pivot

,Póliza,Catastrófico,Dental,Salud,Vida
Nombre,RUT,,,,
Ana López,23456789-0,1.2,0.0,2.5,2.0
Carlos Torres,34567890-1,1.5,1.0,3.2,1.8
Juan Pérez,12345678-9,0.0,0.8,3.0,1.5
Luis Fernández,56789012-3,1.4,0.0,3.1,1.7
María González,45678901-2,0.0,0.9,0.0,1.9


In [5]:
# Paso 6: Comparar con el listado de desvinculados
# Supongamos que cargamos un archivo con los desvinculados
# Este archivo debe contener al menos las columnas "Nombre" y "RUT"
file_path_desvinculados = "Desvinculados.xlsx"  # Subir este archivo a Colab
df_desvinculados = pd.read_excel(file_path_desvinculados)

# Encontrar trabajadores desvinculados que aún aparecen en las pólizas
trabajadores_activos = set(zip(df_consolidado["Nombre"], df_consolidado["RUT"]))
trabajadores_desvinculados = set(zip(df_desvinculados["Nombre"], df_desvinculados["RUT"]))

# Identificar inconsistencias
desvinculados_en_polizas = trabajadores_desvinculados & trabajadores_activos
correctamente_eliminados = trabajadores_desvinculados - trabajadores_activos

# Paso 7: Exportar resultados
# Guardar resumen de pólizas y inconsistencias en un nuevo archivo Excel
output_path = "Resumen_Polizas_Seguros.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_pivot.to_excel(writer, sheet_name="Resumen Pólizas")
    pd.DataFrame(list(desvinculados_en_polizas), columns=["Nombre", "RUT"]).to_excel(writer, sheet_name="Desvinculados en Pólizas", index=False)
    pd.DataFrame(list(correctamente_eliminados), columns=["Nombre", "RUT"]).to_excel(writer, sheet_name="Correctamente Eliminados", index=False)

print(f"Proceso completado. Los resultados se han guardado en {output_path}")

Proceso completado. Los resultados se han guardado en Resumen_Polizas_Seguros.xlsx
